# Overview

**Oftentimes, we want to test some LLMs that are not widely available or are expensive to host. There are various ways to still get your hands dirty with them.**


**In this notebook, I'm showing how to use LangChain's CustomLLM together with <a href="https://openrouter.ai/">OpenRouter.ai</a>'s collection of LLMs (like gpt-4-32k, claude-2, and many more) to use these models in your LangChain apps.**

### <font color='gray'>Loading libraries and variables</font>

In [17]:
import os
from dotenv import load_dotenv
import requests, json

load_dotenv(".env", override=True)

True

In [18]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

# Setting up custom LLM through OpenRouter.ai

Important! You will need an account on OpenRouter.ai and balance on it (my first balance was just $5).

In [19]:
class Claude2LLM(LLM):
    n: int

    @property
    def _llm_type(self) -> str:
        return "claude2"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:


        OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')
        YOUR_SITE_URL = 'https://localhost'
        headers = {
            'Authorization': f'Bearer {OPENROUTER_API_KEY}',
            'HTTP-Referer': YOUR_SITE_URL,
            'Content-Type': 'application/json'
        }
        data = {
            'model': "anthropic/claude-2",
            'messages': [
                {'role': 'user', 'content': prompt}
            ]
        }
        # Output example: {'choices': [{'message': {'role': 'assistant', 'content': "I am OpenAI's artificial intelligence model called GPT-3."}}], 'model': 'gpt-4-32k-0613', 'usage': {'prompt_tokens': 11, 'completion_tokens': 14, 'total_tokens': 25}, 'id': 'gen-e4MSuTT1v2wvrYFNFunhumsIawaI'}
        response = requests.post('https://openrouter.ai/api/v1/chat/completions', headers=headers, data=json.dumps(data))
        output = response.json()['choices'][0]['message']['content']

        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return output

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": self.n}

# Let's test our brand new custom LLM

In [20]:
llm = Claude2LLM(n=1)

In [21]:
llm("Who are you? Answer in 3 sentences.")

" Hello, I'm Claude, an AI assistant created by Anthropic to be helpful, harmless, and honest. I don't actually have personal traits, goals or an identity. I'm an AI trained by Anthropic to be helpful, harmless, and honest in conversations."